# AAFormer: Adaptive Agent Transformer for Few-shot Segmentation

**Paper authors:** Yuan Wang, Rui Sun, Zhe Zhang, Tianzhu Zhang

**Reproduced by:** Yusuf Soydan, Bartu Akyürek

This paper is about few-shot segmentation (FSS), aiming to segment objects in a given image and a set of support images with masks.

## Prepare Workspace


In [1]:
# General Dependencies

import os
import math
import numpy as np
import torch


In [2]:
"""
Run from Colab or local
"""

try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    ROOT_PATH = "/content/gdrive/MyDrive/AAFormer"
    DATA_PATH = os.path.join(ROOT_PATH, "Datasets")
    
    %cd ./gdrive/MyDrive/AAFormer

except:
    ROOT_PATH = os.curdir
    DATA_PATH = "../Datasets"




In [3]:
# Source dependencies
from data.dataset import FSSDataset
from common.vis import Visualizer
from common.evaluation import Evaluator

## Prepare PASCAL-$5^i$ Dataset

In [4]:
# STEP 1: Download PASCAL VOC2012 devkit (train/val data): (uncomment lines below)
# ------------------------------------------------------------------------------
#!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
#!tar -xvf 'VOCtrainval_11-May-2012.tar' -C ./Datasets/ 

# (or instead of wget, use directly the link to download)
# STEP 2: Place "VOC2012" folder from downloaded "VOCdevkit" folder under a "Datasets" folder.
# ------------------------------------------------------------------------------
#!mv Datasets/VOCdevkit/VOC2012 Datasets/VOC2012

# STEP 3: Download extended annotations from here
#!wget https://drive.google.com/file/d/10zxG2VExoEZUeyQl_uXga2OWHjGeZaf2/view

# STEP 4: Extract Put the downloaded extension under "Datasets/VOC2012/"


In [5]:
# Dataset initialization 
# TODO: does paper mention about image size? TODO: yes, 473
# TODO: they also do data aug...
# TODO: add disclaimer to dataset files https://github.com/juhongm999/hsnet/blob/main/train.py
FSSDataset.initialize(img_size=128, datapath=DATA_PATH, use_original_imgsize=False)
dataloader_trn = FSSDataset.build_dataloader(benchmark='pascal', bsz=4, nworker=1, fold=0, split='trn')
dataloader_val = FSSDataset.build_dataloader(benchmark='pascal', bsz=4, nworker=1, fold=0, split='val')

Total (trn) images are : 11394
Total (val) images are : 346


## Initialize Models

In [11]:
from model.tokens import init_agent_tokens
from model.featureextractor import FeatureExtractor
from model.representationencoder import RepresentationEncoder
from model.learningdecoder import AgentLearningDecoder

# HYPERPARAMETERS
# -----------------------------------------------------------------
# Note: the parameters with * comment are not provided by the paper
# TODO: make hyperparameters UPPER_CASE
reduce_dim = 256
c = 2048        # * Hidden dimensions
hw = 256        # * Dimensionality of Feature maps (depends on ResNet choice)
N = 16          # * Number of layers in Encoder/Decoder
heads = 8       # * Number of attention heads
num_tokens = 15 # * Number of Agent Tokens
sinkhorn_reg = 5e-1 # * Regularization term of Optimal Transport

# Additional parameters to control flow of the model
# ----------------------------------------------------------------
bypass_ot = False # To turn off OT module if it causes any problem

cuda = torch.cuda.is_available() 

# Models of AAFormer 
feature_extractor = FeatureExtractor(layers=50, reduce_dim=reduce_dim, c=c)
representation_encoder = RepresentationEncoder(c, hw, N, heads)
agent_learning_decoder = AgentLearningDecoder(cuda, c, N, num_tokens, sinkhorn_reg = sinkhorn_reg)


## Training Loop

In [13]:
for idx, batch in enumerate(dataloader_trn):
    
    # STEP 0: Get query image and support images with corresponding masks
    query_img = batch['query_img']
    supp_imgs = batch['support_imgs']
    supp_masks = batch['support_masks']
    
    ## TODO: Start of AAFormer() -------------------------------------------------------------
    
    # STEP 1: Extract Features from the backbone model (ResNet)
    F_Q, F_S, s_mask_list, f_s, M_s = feature_extractor(query_img, supp_imgs, supp_masks)

    # STEP 2.1: Pass the features from encoder 
    F_S_hat = representation_encoder(F_S)
    F_Q_hat = representation_ecoder(F_Q)
    
    # STEP 2.2: Get Initial Agent Tokens
    # TODO: can we get rid of for loop?
    # Since every image will have different number of foreground pixels, it is not possible
    # to combine the foreground pixels of images in a single tensor. We may pad the tensors
    # since max number of foreground pixels is equal to the image area.
    X, L = [], []
    # TODO: This part assumes we are doing 1-shot learning
    for i, m in enumerate(M_s):  # M_s has shape (batchsize, 1, 16, 16)
      m = m.squeeze(0)  # Get a single mask, shape (16, 16)

      fg = np.where(m == 1.) # get foreground pixels
      bg = np.where(m == 0.) # get background pixels
      
      # Create tensor with shape [num_foreground_pix, 2] where the last dimension has (x,y) locations of foreground pixels
      foreground_pix = torch.stack((torch.from_numpy(fg[0]), torch.from_numpy(fg[1])), dim=1)
      background_pix = torch.stack((torch.from_numpy(bg[0]), torch.from_numpy(bg[1])), dim=1)

      X.append(foreground_pix)
      L.append(background_pix)

    agent_tokens = init_agent_tokens(num_tokens, X, L, f_s) # every token has [K,c] dim for every sample in a batch
    
    # STEP 3: Pass initial agent tokens through Agent Learning Decoder and obtain agent tokens.
    # Note: agent_tokens has shape (batchsize, num_tokens, c)
    agent_tokens = agent_learning_decoder(tokens, F_S_hat, M_s, bypass_ot=bypass_ot)
    
    print(agent_tokens.shape)
    
    # STEP 4: Pass agent tokens through Agent Matching Decoder
    # F_S_hat, F_Q_hat,...
    
    ## TODO: End of AAFormer() -------------------------------------------------------------
    ## TODO: Move above part to a class AAFormer-->forward() and get the predictions from it.
    
    # TODO: compute loss
    # TODO: optimizer...
    # TODO: complete training loop
    break

torch.Size([4, 15, 2048])
torch.Size([4, 15, 2048])


## Visualization Test

In [7]:
""" Code below visualizes prediction (to be used in testing)
Visualizer.initialize(True)
Evaluator.initialize()

for idx, batch in enumerate(dataloader_trn):
        # 1. Hypercorrelation Squeeze Networks forward pass
        #batch = utils.to_cuda(batch)
        #pred_mask = model.module.predict_mask_nshot(batch, nshot=nshot)

        #assert pred_mask.size() == batch['query_mask'].size()
        pred_mask = batch['query_mask']

        # 2. Evaluate prediction
        area_inter, area_union = Evaluator.classify_prediction(pred_mask.clone(), batch)
        
        #average_meter.update(area_inter, area_union, batch['class_id'], loss=None)
        #average_meter.write_process(idx, len(dataloader), epoch=-1, write_batch_idx=1)

        # Visualize predictions
        if Visualizer.visualize:
            Visualizer.visualize_prediction_batch(batch['support_imgs'], batch['support_masks'],
                                                  batch['query_img'], batch['query_mask'],
                                                  pred_mask, batch['class_id'], idx,
                                                  area_inter[1].float() / area_union[1].float())

        break   # TODO: delete this break to run visualization for full dataset
"""

" Code below visualizes prediction (to be used in testing)\n\nfor idx, batch in enumerate(dataloader_trn):\n        # 1. Hypercorrelation Squeeze Networks forward pass\n        #batch = utils.to_cuda(batch)\n        #pred_mask = model.module.predict_mask_nshot(batch, nshot=nshot)\n\n        #assert pred_mask.size() == batch['query_mask'].size()\n        pred_mask = batch['query_mask']\n\n        # 2. Evaluate prediction\n        area_inter, area_union = Evaluator.classify_prediction(pred_mask.clone(), batch)\n        \n        #average_meter.update(area_inter, area_union, batch['class_id'], loss=None)\n        #average_meter.write_process(idx, len(dataloader), epoch=-1, write_batch_idx=1)\n\n        # Visualize predictions\n        if Visualizer.visualize:\n            Visualizer.visualize_prediction_batch(batch['support_imgs'], batch['support_masks'],\n                                                  batch['query_img'], batch['query_mask'],\n                                          

In [ ]:
# TODO: declare numerical results